In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("audit_5%.csv")

In [3]:
df

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,...,NaN,0.0,0.2,0.0,2.4,8.574,0.4,0.5,NaN,1
1,NaN,0.00,NaN,0.000,4.83,0.2,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0
3,3.89,0.00,0.2,NaN,10.80,0.6,6.480,NaN,6.0,0.6,...,0.4,0.0,0.2,NaN,4.4,17.530,0.4,0.5,3.5060,1
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,NaN,0.2,...,0.4,NaN,NaN,0.0,2.0,1.416,0.4,0.5,0.2832,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,55.57,1.28,0.4,0.512,0.00,0.2,0.000,1.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.2,1.956,0.4,0.5,0.3912,0
678,55.57,0.00,0.2,0.000,0.00,0.2,0.000,0.00,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.400,0.4,0.5,0.2800,0
679,55.57,0.00,0.2,0.000,0.00,0.2,0.000,0.00,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.400,0.4,0.5,0.2800,0
680,55.57,0.21,0.2,0.042,0.00,0.2,0.000,0.21,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.442,0.4,0.5,0.2884,0


In [4]:
df.dtypes

Sector_score      float64
PARA_A            float64
Score_A           float64
Risk_A            float64
PARA_B            float64
Score_B           float64
Risk_B            float64
TOTAL             float64
numbers           float64
Score_B.1         float64
Risk_C            float64
Money_Value       float64
Score_MV          float64
Risk_D            float64
District_Loss     float64
PROB              float64
RiSk_E            float64
History           float64
Prob              float64
Risk_F            float64
Score             float64
Inherent_Risk     float64
CONTROL_RISK      float64
Detection_Risk    float64
Audit_Risk        float64
Risk                int64
dtype: object

In [5]:
df.isnull().sum()

Sector_score      49
PARA_A            40
Score_A           36
Risk_A            41
PARA_B            39
Score_B           36
Risk_B            41
TOTAL             27
numbers           35
Score_B.1         24
Risk_C            36
Money_Value       41
Score_MV          32
Risk_D            28
District_Loss     21
PROB              42
RiSk_E            40
History           41
Prob              41
Risk_F            38
Score             33
Inherent_Risk     32
CONTROL_RISK      34
Detection_Risk    35
Audit_Risk        42
Risk               0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 17732
nilai hilang : 904
persentase : 5.0981276787728405


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Sector_score      7.184751
PARA_A            5.865103
Score_A           5.278592
Risk_A            6.011730
PARA_B            5.718475
Score_B           5.278592
Risk_B            6.011730
TOTAL             3.958944
numbers           5.131965
Score_B.1         3.519062
Risk_C            5.278592
Money_Value       6.011730
Score_MV          4.692082
Risk_D            4.105572
District_Loss     3.079179
PROB              6.158358
RiSk_E            5.865103
History           6.011730
Prob              6.011730
Risk_F            5.571848
Score             4.838710
Inherent_Risk     4.692082
CONTROL_RISK      4.985337
Detection_Risk    5.131965
Audit_Risk        6.158358
Risk              0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        x1 = np.nan_to_num(x1)  # Mengganti nilai NaN dengan 0
        x2 = np.nan_to_num(x2)  # Mengganti nilai NaN dengan 0
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",dist)

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter


def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,9)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [11]:
df2 = df.copy()
dummy = df2.fillna(0)
# dummy.fillna(0, inplace=True)
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,...,0.0,0.0,0.2,0.0,2.4,8.574,0.4,0.5,0.0000,1
1,0.00,0.00,0.0,0.000,4.83,0.2,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,0.00,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,0.0,0.2,...,0.4,0.0,0.0,0.0,2.0,1.416,0.4,0.5,0.2832,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,55.57,1.28,0.4,0.512,0.00,0.2,0.000,1.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.2,1.956,0.4,0.5,0.3912,0
678,55.57,0.00,0.2,0.000,0.00,0.2,0.000,0.00,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.400,0.4,0.5,0.2800,0
679,55.57,0.00,0.2,0.000,0.00,0.2,0.000,0.00,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.400,0.4,0.5,0.2800,0
680,55.57,0.21,0.2,0.042,0.00,0.2,0.000,0.21,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.442,0.4,0.5,0.2884,0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 3701


In [13]:
euc_dist = {}

In [14]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,9)

Kolom : 1
[0.0, 2.37, 2.37, 3.89, 3.89, 3.89, 55.57, 55.57, 55.57]
min :  0.0
max :  55.57
mean :  20.346666666666668
fuzzy :  [(3.89, 0.19118610747051112), (3.89, 0.19118610747051112), (2.37, 0.11648099606815203), (55.57, 0.0), (2.37, 0.11648099606815203), (3.89, 0.19118610747051112), (55.57, 0.0), (55.57, 0.0), (0.0, 0.0)]
fuzzy :  None
pembilang :  2.783261795543906
penyebut :  0.8065203145478375
rata-rata tertimbang :  3.450950639853748
 
[2.72, 2.72, 2.72, 3.41, 3.41, 3.41, 3.41, 3.41, 3.89]
min :  2.72
max :  3.89
mean :  3.233333333333333
fuzzy :  [(2.72, 0.0), (3.89, 0.0), (3.41, 0.7309644670050756), (3.41, 0.7309644670050756), (3.41, 0.7309644670050756), (2.72, 0.0), (2.72, 0.0), (3.41, 0.7309644670050756), (3.41, 0.7309644670050756)]
fuzzy :  None
pembilang :  12.462944162436537
penyebut :  3.654822335025378
rata-rata tertimbang :  3.4099999999999997
 
[0.0, 2.72, 2.72, 3.41, 3.41, 3.41, 3.89, 3.89, 3.89]
min :  0.0
max :  3.89
mean :  3.037777777777778
fuzzy :  [(2.72, 0.895

[0.0, 0.0, 1.99, 1.99, 2.72, 3.89, 55.57, 55.57, 55.57]
min :  0.0
max :  55.57
mean :  19.7
fuzzy :  [(55.57, 0.0), (1.99, 0.10101522842639594), (2.72, 0.13807106598984772), (55.57, 0.0), (55.57, 0.0), (0.0, 0.0), (1.99, 0.10101522842639594), (3.89, 0.19746192893401016), (0.0, 0.0)]
fuzzy :  None
pembilang :  1.5457208121827413
penyebut :  0.5375634517766498
rata-rata tertimbang :  2.875420207743154
 
[0.0, 2.37, 2.37, 2.37, 2.37, 55.57, 55.57, 55.57, 55.57]
min :  0.0
max :  55.57
mean :  25.75111111111111
fuzzy :  [(0.0, 0.0), (55.57, 0.0), (2.37, 0.09203486365205385), (55.57, 0.0), (55.57, 0.0), (2.37, 0.09203486365205385), (55.57, 0.0), (2.37, 0.09203486365205385), (2.37, 0.09203486365205385)]
fuzzy :  None
pembilang :  0.8724905074214705
penyebut :  0.3681394546082154
rata-rata tertimbang :  2.37
 
[0.0, 1.85, 2.37, 2.37, 3.577731958762887, 3.89, 55.57, 55.57, 55.57]
min :  0.0
max :  55.57
mean :  20.085303550973652
fuzzy :  [(0.0, 0.0), (55.57, 0.0), (3.89, 0.19367394623276327)

[0.0, 2.37, 2.37, 20.79890614599512, 21.61, 21.61, 21.61, 21.61, 55.57]
min :  0.0
max :  55.57
mean :  18.616545127332795
fuzzy :  [(0.0, 0.0), (21.61, 0.9189939105022267), (21.61, 0.9189939105022267), (2.37, 0.12730611312624104), (2.37, 0.12730611312624104), (20.79890614599512, 0.9409429774243783), (55.57, 0.0), (21.61, 0.9189939105022267), (21.61, 0.9189939105022267)]
fuzzy :  None
pembilang :  99.61184927621369
penyebut :  4.8715308456857676
rata-rata tertimbang :  20.44775090861429
 
[0.0, 0.0, 2.37, 2.6198277014429743, 2.72, 3.41, 3.89, 3.89, 3.89]
min :  0.0
max :  3.89
mean :  2.5322030779381084
fuzzy :  [(0.0, 0.0), (3.89, 0.0), (2.6198277014429743, 0.9354655898234009), (2.72, 0.861689978073664), (3.41, 0.35351383715842627), (3.89, 0.0), (3.89, 0.0), (2.37, 0.9359438903809465), (0.0, 0.0)]
fuzzy :  None
pembilang :  8.21822461123948
penyebut :  3.086613295436438
rata-rata tertimbang :  2.6625378123622214
 
[0.0, 0.0, 1.85, 3.41, 3.41, 3.89, 55.57, 55.57, 59.85]
min :  0.0
max 

[0.0, 0.0, 0.73, 0.76, 0.77, 0.7783111111111111, 0.84, 1.52, 2.13]
min :  0.0
max :  2.13
mean :  0.8364790123456791
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.84, 0.9972779818124902), (0.76, 0.9085703153134537), (2.13, 0.0), (1.52, 0.47158106116714643), (0.7783111111111111, 0.9304610153081404), (0.77, 0.9205251878833676), (0.73, 0.8727056976037121)]
fuzzy :  None
pembilang :  4.315097857925733
penyebut :  5.10112125908831
rata-rata tertimbang :  0.8459116415313253
 
[0.0, 0.0, 0.05, 1.6, 1.71, 1.82, 2.95, 3.95, 7.18]
min :  0.0
max :  7.18
mean :  2.14
fuzzy :  [(0.05, 0.02336448598130841), (0.0, 0.0), (1.82, 0.8504672897196262), (1.71, 0.7990654205607476), (7.18, 0.0), (2.95, 0.8392857142857143), (3.95, 0.6408730158730159), (1.6, 0.7476635514018691), (0.0, 0.0)]
fuzzy :  None
pembilang :  9.119023512831925
penyebut :  3.9007194778222813
rata-rata tertimbang :  2.3377798800140717
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.51, 0.5441269841269841, 0.62]
min :  0.0
max :  0.62
mean :  0.18601410934744

[0.0, 0.01, 1.03, 1.14, 1.19, 1.628166250030635, 2.19, 3.43, 7.59]
min :  0.0
max :  7.59
mean :  2.0231295833367375
fuzzy :  [(1.14, 0.5634834315060548), (3.43, 0.7472780375034257), (7.59, 0.0), (2.19, 0.9700243756054084), (1.628166250030635, 0.8047760575698313), (0.01, 0.0049428371184741665), (0.0, 0.0), (1.19, 0.5881976170984258), (1.03, 0.5091122232028391)]
fuzzy :  None
pembilang :  7.864587561514644
penyebut :  4.187814579604459
rata-rata tertimbang :  1.8779693828415531
 
[0.0, 0.0, 0.0, 0.0, 0.383886751410025, 0.42, 0.4415035106694052, 0.7783111111111111, 1.0]
min :  0.0
max :  1.0
mean :  0.33596681924339344
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.42, 0.873450328700656), (0.4415035106694052, 0.8410671416965007), (0.7783111111111111, 0.3338521256366952), (0.0, 0.0), (1.0, 0.0), (0.383886751410025, 0.9278350336167974), (0.0, 0.0)]
fuzzy :  None
pembilang :  1.3542076295726289
penyebut :  2.976204629650649
rata-rata tertimbang :  0.45501159970024896
 
[0.0, 0.0, 0.17, 0.383886751410

...

[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), 

[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), 

[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), 

[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), 

[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 0, 0, 0, 0]
median :  5
min :  0
max :  0
mean :  0.0
fuzzy :  [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), 

In [15]:
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.890000,4.180000,0.6,2.508000,2.50,0.2,0.500,6.680000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.4,8.574,0.4,0.5,1.915576,1
1,3.450951,1.025145,0.2,0.193762,4.83,0.2,0.966,4.830000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.510800,0
2,3.890000,0.510000,0.2,0.102000,0.23,0.2,0.046,0.740000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.309600,0
3,3.890000,1.858113,0.2,0.812547,10.80,0.6,6.480,12.283805,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.506000,1
4,3.890000,0.872384,0.2,0.276441,0.08,0.2,0.016,0.080000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.416,0.4,0.5,0.283200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,55.570000,1.280000,0.4,0.512000,0.00,0.2,0.000,1.280000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.2,1.956,0.4,0.5,0.391200,0
678,55.570000,0.000000,0.2,0.000000,0.00,0.2,0.000,0.000000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.400,0.4,0.5,0.280000,0
679,55.570000,0.000000,0.2,0.000000,0.00,0.2,0.000,0.000000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.400,0.4,0.5,0.280000,0
680,55.570000,0.210000,0.2,0.042000,0.00,0.2,0.000,0.210000,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.442,0.4,0.5,0.288400,0


In [16]:
dummy.describe()

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,...,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.0,682.000000,682.000000
mean,14.680333,2.567110,0.367808,1.542189,12.192558,0.322255,7.175841,14.824046,5.075993,0.226581,...,0.527164,0.114370,0.219276,0.059824,2.761225,19.757987,0.591530,0.5,7.810666,0.439883
std,21.260848,4.970012,0.176685,3.618232,53.199613,0.173410,31.984426,54.546352,0.276766,0.083800,...,0.292948,0.561986,0.072276,0.325065,0.869422,58.046246,0.465162,0.0,40.764796,0.496737
min,1.850000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.000000,1.400000,0.400000,0.5,0.280000,0.000000
25%,2.370000,0.370000,0.200000,0.086224,0.000000,0.200000,0.000000,0.552500,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.000000,1.638500,0.400000,0.5,0.324800,0.000000
50%,3.410000,1.050000,0.400000,0.440000,0.490000,0.200000,0.098000,1.595000,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.400000,2.420000,0.400000,0.5,0.757400,0.000000
75%,21.610000,3.095000,0.600000,1.879500,5.542500,0.600000,2.809500,9.420000,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,3.500226,15.485500,0.567442,0.5,3.983500,1.000000
max,59.850000,85.000000,0.600000,51.000000,1264.630000,0.600000,758.778000,1268.910000,9.000000,0.600000,...,2.400000,9.000000,0.600000,5.400000,5.200000,801.262000,5.800000,0.5,961.514400,1.000000


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
x = dummy.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [20]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [21]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[100   1]
 [  1  69]]
0.9883040935672515
